# Energy Industry Overview

{badge}`python3,badge-success` {badge}`case study,badge-secondary` {badge}`electricity market,badge-warning` 

Working with these files can be a challenge, especially given their heterogeneous nature. Some preprocessing is required before they are ready for consumption by your CNN.

Fortunately, I participated in the LUNA16 competition as part of a university course on computer aided diagnosis, so I have some experience working with these files. At this moment we top the leaderboard there :)

**This article provides an exploratory data analysis on the Philippine electricity market by comparing the data provided by IEMOP and DOE on their websites.**

What we will cover:

1. **Pre-processing:** and adding missing metadata
2. **Exploratory Data Analysis:** and what tissue these unit values correspond to
3. **Resampling** to an isomorphic resolution to remove variance in scanner resolution.

**Dataset:** The HR Dataset was designed by Drs. Rich Huebner and Carla Patalano to accompany a case study designed for graduate HR students studying HR metrics, measurement, and analytics. The students use Tableau data visualization software to uncover insights about the case. This is a synthetic data set created specifically to go along with the case study (proprietary for the college that we teach at). https://rpubs.com/rhuebner/hr_codebook_v14

***


## Data Extraction

The data comes from the published market participants list by IEMOP, the electricity market operator of the Philippines.

Cache the data; check if extracted is equal to imported, if not get the difference, then run the geocoding

In [329]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt

In [330]:
# Formatting of DataFrame html view
t_props = [
  ('font-size', '80%')
  ]
   
styles = [
  dict(selector="th", props=t_props),
  dict(selector="td", props=t_props)
  ]

In [331]:
def get_data(url):
    html_data = pd.read_html(url)
    data = html_data[0]
    return data

In [332]:
df_wesm = get_data('http://www.iemop.ph/the-market/participants/wesm-members/')
# Dropping 451 Team Sual NA Resource value
df_wesm = df_wesm.dropna()

In [333]:
df_retail = get_data('http://www.iemop.ph/the-market/participants/rcoa-tp/')
df_retail = df_retail.dropna()

In [334]:
# load a simple dataset as a pandas DataFrame
from vega_datasets import data
cars = data.cars()

alt.Chart(cars).mark_point().encode(
    x='Horsepower',
    y='Miles_per_Gallon',
    color='Origin',
)

alt.Chart(...)

In [335]:
df_wesm.describe()

,PARTICIPANT NAME,SHORT NAME,REGION,CATEGORY,MEMBERSHIP,RESOURCE,EFFECTIVE DATE,STATUS
count,454,454,454,454,454,454,454,454
unique,308,320,4,5,2,443,245,4
top,Power Sector Assets & Liabilities Management C...,PSALM,LUZON,GENERATOR,DIRECT,Palauig Solar Power Plant,12/26/2010,REGISTERED
freq,20,20,277,292,388,3,59,420


In [336]:
df_retail.describe()

,NAME,SHORT NAME,REGION,CATEGORY,MEMBERSHIP,EFFECTIVE DATE,STATUS
count,1668,1668,1668,1668,1668,1668,1668
unique,1457,1668,3,6,3,199,2
top,Megaworld Corporation,LXMRK2R,Luzon,Contestable Customer,Indirect,26-Jun-13,Registered
freq,12,1,1443,1507,1537,222,1622


In [337]:
df_wesm.dtypes

PARTICIPANT NAME    object
SHORT NAME          object
REGION              object
CATEGORY            object
MEMBERSHIP          object
RESOURCE            object
EFFECTIVE DATE      object
STATUS              object
dtype: object

### Exploratory Data Analysis

We explore the dataset by looking at each column; looking at unique values, calculating summaries, and plotting distributions.

### Wholesale Electricity Spot Market

This section discusses the data on Wholesale Electricity Spot Market participants.

First, we look at the column names to see the variables from the data. We get the counts of the unique values in each column.

#### Participant Name and Short Name

From the counts above, we notice that `PARTICIPANT NAME` and `SHORT NAME` are not equal. This means that there are duplicate values for these variables. We investigate by performing a cross-tabulation.

In [338]:
df_cross = pd.crosstab(df_wesm['SHORT NAME'], df_wesm['PARTICIPANT NAME'],margins=True)
df_cross.tail().style.set_table_styles(styles)

Based on the tabulation, we can see that there are `PARTICIPANT NAME` with multiple `SHORT NAME`.

In [339]:
df_cross.All[(df_cross.All > 1)].head(n=10)


SHORT NAME
AFAB                  2
AHC                   2
APRI                 10
BGI                   2
CEDC                  3
CENPRI                4
EDC                   3
FDC                   3
FGHPC                 3
FGP - San Lorenzo     2
Name: All, dtype: int64

We take a look at some of the values.

In [340]:
df_wesm[df_wesm['SHORT NAME'] =='APRI'].tail().style.set_table_styles(styles)

,PARTICIPANT NAME,SHORT NAME,REGION,CATEGORY,MEMBERSHIP,RESOURCE,EFFECTIVE DATE,STATUS
62,AP Renewables Inc.,APRI,LUZON,GENERATOR,DIRECT,Tiwi Geothermal Power Plant A,05/26/2009,REGISTERED
63,AP Renewables Inc.,APRI,LUZON,GENERATOR,DIRECT,Tiwi Geothermal Power Plant B,06/15/2019,DEREGISTERED
64,AP Renewables Inc.,APRI,LUZON,GENERATOR,DIRECT,Makban Geothermal Power Plant E,05/26/2009,REGISTERED
65,AP Renewables Inc.,APRI,LUZON,GENERATOR,DIRECT,Tiwi Geothermal Power Plant C,05/26/2009,REGISTERED
395,AP Renewables Inc.,APRI,LUZON,GENERATOR,DIRECT,Makban Geothermal Power Plant B,06/15/2019,REGISTERED


In [341]:
df_wesm[df_wesm['SHORT NAME'] =='AHC'].tail().style.set_table_styles(styles)

,PARTICIPANT NAME,SHORT NAME,REGION,CATEGORY,MEMBERSHIP,RESOURCE,EFFECTIVE DATE,STATUS
268,Angat Hydropower Corporation,AHC,LUZON,GENERATOR,DIRECT,Angat Hydro Electric Power Plant Unit A,10/31/2014,REGISTERED
269,Angat Hydropower Corporation,AHC,LUZON,GENERATOR,DIRECT,Angat Hydro Electric Power Plant Unit M,10/31/2014,REGISTERED


We can see that each `PARTICIPANT NAME` has multiple `RESOURCE` entries. From the inspected data above, we can infer that membership in the spot market is a disaggregation of the power plant units of each participating company.

#### Region

We look at the `REGION` variable and see that most of the participants are from Luzon.

In [342]:
df_wesm.groupby('REGION')['RESOURCE'].nunique()


REGION
LUZON              269
LUZON / VISAYAS      9
MINDANAO            38
VISAYAS            128
Name: RESOURCE, dtype: int64

In [343]:
df_retail.groupby('REGION')['NAME'].nunique()


REGION
Luzon            1253
Luzon/Visayas      34
Visayas           176
Name: NAME, dtype: int64

We look at the participants with dual regions.

In [344]:
df_wesm[df_wesm['REGION'] =='LUZON / VISAYAS'].tail().style.set_table_styles(styles)

,PARTICIPANT NAME,SHORT NAME,REGION,CATEGORY,MEMBERSHIP,RESOURCE,EFFECTIVE DATE,STATUS
263,National Grid Corporation of the Philippines,NGCP,LUZON / VISAYAS,BULK USER,DIRECT,National Grid Corporation of the Philippines,07/29/2014,REGISTERED
358,National Power Corporation,NPC,LUZON / VISAYAS,GENERATOR,DIRECT,Power Barges,09/25/2016,CEASED
428,Manta Energy Inc.,MEI,LUZON / VISAYAS,WAG,DIRECT,Manta Energy Inc.,01/26/2020,CEASED
429,Trans-Asia Oil and Development Corporation,TAO,LUZON / VISAYAS,WAG,DIRECT,Trans-Asia Oil and Development Corporation,01/26/2020,CEASED
431,Angeles Power Inc.,API,LUZON / VISAYAS,WAG,DIRECT,Angeles Power Inc.,02/26/2020,CEASED


In [345]:
df_retail[df_retail['REGION'] =='Luzon/Visayas']

,NAME,SHORT NAME,REGION,CATEGORY,MEMBERSHIP,EFFECTIVE DATE,STATUS
101,GNPower Ltd. Co.,GNPLCRES,Luzon/Visayas,Retail Electricity Supplier,Direct,15-Apr-14,Registered
171,Waterfront Mactan Casino Hotel Inc.,WAHCRES,Luzon/Visayas,Retail Electricity Supplier,Direct,26-Oct-15,Registered
179,GNPower Mariveles Coal Plant Ltd. Co.,GNPRES,Luzon/Visayas,Retail Electricity Supplier,Direct,15-Mar-16,Ceased
220,Millennium Power RES Inc.,MPRIRES,Luzon/Visayas,Retail Electricity Supplier,Direct,18-Aug-16,Registered
253,FDC Retail Electricity Sales Corporation,FDCRESC,Luzon/Visayas,Retail Electricity Supplier,Direct,04-Oct-16,Registered
254,AC Energy Inc. (formerly AC Energy Holdings Inc.),ACERES,Luzon/Visayas,Retail Electricity Supplier,Direct,21-Oct-16,Registered
298,Manta Energy Inc.,MANTARES,Luzon/Visayas,Retail Electricity Supplier,Direct,02-Dec-16,Registered
299,SMC Consolidated Power Corporation,SMCCPCRES,Luzon/Visayas,Retail Electricity Supplier,Direct,02-Dec-16,Registered
300,Corenergy Inc.,CORERES,Luzon/Visayas,Retail Electricity Supplier,Direct,23-Dec-16,Registered
370,KEPCO SPC Power Corporation,KSPCRES,Luzon/Visayas,Retail Electricity Supplier,Direct,02-Feb-17,Registered


In [346]:
df_retail[df_retail['REGION'] =='Visayas']

,NAME,SHORT NAME,REGION,CATEGORY,MEMBERSHIP,EFFECTIVE DATE,STATUS
32,Balamban Enerzone Corporation,BEZMSP,Visayas,Retail Metering Services Provider,Service Provider,25-Sep-13,Registered
33,Mactan Enerzone Corporation,MEZMSP,Visayas,Retail Metering Services Provider,Service Provider,25-Sep-13,Registered
94,Cebu II Electric Cooperative Inc.,CEBEC2LRE,Visayas,Local Retail Electricity Supplier,Direct,20-Mar-14,Registered
108,San Miguel Corporation,SMC501R,Visayas,Contestable Customer,Indirect,26-May-14,Registered
129,Central Negros Electric Cooperative Inc.,CENECOLRE,Visayas,Local Retail Electricity Supplier,Direct,19-Aug-14,Registered
...,...,...,...,...,...,...,...
1425,Visayan Electric Company Inc.,VECOLRE,Visayas,Local Retail Electricity Supplier,Direct,19-Jun-13,Registered
1637,Masterpiece Asia Properties Inc. (Vistamall Ta...,MPAP14R,Visayas,Contestable Customer,Indirect,26-Aug-20,Registered
1665,Dynasty Management and Development Corp. (Gais...,DMDC06R,Visayas,Contestable Customer,Indirect,26-Oct-20,Registered
1666,Sunpride Foods Inc.,SNPFI1R,Visayas,Contestable Customer,Indirect,26-Oct-20,Registered


Except for the NGCP, which is a transmission company and NPC, all the participants with dual regions are Aggregators (WAG) with 'ceased' status. ERC has issued a circular to stop the operation of all WAGs.

In [347]:
df_wesm = df_wesm[df_wesm['REGION']!='LUZON / VISAYAS']

In [348]:
df_wesm.describe()

,PARTICIPANT NAME,SHORT NAME,REGION,CATEGORY,MEMBERSHIP,RESOURCE,EFFECTIVE DATE,STATUS
count,445,445,445,445,445,445,445,445
unique,300,311,3,4,2,434,239,4
top,Power Sector Assets & Liabilities Management C...,PSALM,LUZON,GENERATOR,DIRECT,Palauig Solar Power Plant,12/26/2010,REGISTERED
freq,20,20,277,291,379,3,59,419


#### Category

Just like any market, we can see that WESM participants are either buyers or sellers of electricity.

In [349]:
df_wesm['CATEGORY'].unique()

array(['GENERATOR', 'PRIVATE DISTRIBUTION UTILITY',
       'ELECTRIC COOPERATIVE', 'BULK USER'], dtype=object)

#### Membership

Participants have either `DIRECT` or `INDIRECT` membership, which pertains to their grid connection status.

In [350]:
df_wesm['MEMBERSHIP'].unique()

array(['DIRECT', 'INDIRECT'], dtype=object)

#### Effective Date

Date when they participated in the market. We convert the data type.

In [351]:
df_wesm = df_wesm.astype({'EFFECTIVE DATE': 'datetime64'})

In [352]:
df_retail = df_retail.astype({'EFFECTIVE DATE': 'datetime64'})

In [353]:
df_wesm

,PARTICIPANT NAME,SHORT NAME,REGION,CATEGORY,MEMBERSHIP,RESOURCE,EFFECTIVE DATE,STATUS
0,GNPower Kauswagan Ltd. Co.,GNPKLCO,MINDANAO,GENERATOR,DIRECT,GNPK Coal Fired Power Plant Unit 4,2020-08-07,REGISTERED
1,GNPower Kauswagan Ltd. Co.,GNPKLCO,MINDANAO,GENERATOR,DIRECT,GNPK Coal Fired Power Plant Unit 3,2020-08-07,REGISTERED
2,FGP Corporation,FGP - San Lorenzo,LUZON,GENERATOR,DIRECT,San Lorenzo Natural Gas Power Plant 5,2006-06-26,REGISTERED
3,FGP Corporation,FGP - San Lorenzo,LUZON,GENERATOR,DIRECT,San Lorenzo Natural Gas Power Plant 6,2006-06-26,REGISTERED
4,First Gas Power Corporation,FGPCSTRA,LUZON,GENERATOR,DIRECT,Sta. Rita Natural Gas Power Plant 1,2006-06-26,REGISTERED
...,...,...,...,...,...,...,...,...
450,FDC Misamis Power Corporation,FDC,MINDANAO,GENERATOR,DIRECT,135 MW Coal Fired Thermal Power Plant Unit 1,2020-10-26,REGISTERED
451,FDC Misamis Power Corporation,FDC,MINDANAO,GENERATOR,DIRECT,135 MW Coal Fired Thermal Power Plant Unit 2,2020-10-26,REGISTERED
452,FDC Misamis Power Corporation,FDC,MINDANAO,GENERATOR,DIRECT,135 MW Coal Fired Thermal Power Plant Unit 3,2020-10-26,REGISTERED
453,Malvar Enerzone Corporation,MALVEZ,LUZON,PRIVATE DISTRIBUTION UTILITY,DIRECT,Malvar Enerzone Corporation,2020-10-30,REGISTERED


#### Status

We only consider participants with `REGISTERED` status

In [354]:
df_wesm['STATUS'].unique()

array(['REGISTERED', 'DEREGISTERED',
       'REGISTERED; WITH NOTICE OF SUSPENSION', 'CEASED'], dtype=object)

In [355]:
df_retail['STATUS'].unique()

array(['Registered', 'Ceased'], dtype=object)

In [356]:
df_wesm = df_wesm[df_wesm['STATUS']=='REGISTERED']
df_retail = df_retail[df_retail['STATUS']=='Registered']


In [357]:
df_wesm['STATUS'].unique()

array(['REGISTERED'], dtype=object)

In [358]:
df_retail['STATUS'].unique()

array(['Registered'], dtype=object)

In [359]:
df_wesm = df_wesm.drop(['STATUS'],axis=1)
df_retail = df_retail.drop(['STATUS'],axis=1)


#### Visualisations

In [360]:
import os
import googlemaps
gmap = googlemaps.Client(key=os.getenv('GOOGLE_API_KEY'))

In [361]:
# Reset index since the rows dropped caused the index to skip integer sequence
df_wesm = df_wesm.reset_index(drop=True)


In [362]:
df_wesm.isnull().sum()

PARTICIPANT NAME    0
SHORT NAME          0
REGION              0
CATEGORY            0
MEMBERSHIP          0
RESOURCE            0
EFFECTIVE DATE      0
dtype: int64

In [363]:
# Geocoding an address
#df_wesm_coded = pd.DataFrame({"PLACE":names})
df_wesm["LAT"] = None
df_wesm["LON"] = None
country_limit = {'country': 'Philippines'}
geocode_result = []

for i in range(0, len(df_wesm.index),1): 
    geocode_result = gmap.geocode(df_wesm['RESOURCE'][i],country_limit)
    
    if geocode_result: #meaning its successful
        pass
    else:
        geocode_result = gmap.geocode(df_wesm['PARTICIPANT NAME'][i] + ' ' + df_wesm['RESOURCE'][i],country_limit)

    try:
        lat = geocode_result[0]["geometry"]["location"]["lat"]
        lon = geocode_result[0]["geometry"]["location"]["lng"]
        df_wesm.iat[i,df_wesm.columns.get_loc("LAT")] = lat
        df_wesm.iat[i,df_wesm.columns.get_loc("LON")] = lon
    except Exception as e:
        #IndexError
        lat = None
        lon = None


In [364]:
df_wesm

,PARTICIPANT NAME,SHORT NAME,REGION,CATEGORY,MEMBERSHIP,RESOURCE,EFFECTIVE DATE,LAT,LON
0,GNPower Kauswagan Ltd. Co.,GNPKLCO,MINDANAO,GENERATOR,DIRECT,GNPK Coal Fired Power Plant Unit 4,2020-08-07,8.18565,124.118
1,GNPower Kauswagan Ltd. Co.,GNPKLCO,MINDANAO,GENERATOR,DIRECT,GNPK Coal Fired Power Plant Unit 3,2020-08-07,8.18565,124.118
2,FGP Corporation,FGP - San Lorenzo,LUZON,GENERATOR,DIRECT,San Lorenzo Natural Gas Power Plant 5,2006-06-26,10.6046,122.684
3,FGP Corporation,FGP - San Lorenzo,LUZON,GENERATOR,DIRECT,San Lorenzo Natural Gas Power Plant 6,2006-06-26,13.7765,121.033
4,First Gas Power Corporation,FGPCSTRA,LUZON,GENERATOR,DIRECT,Sta. Rita Natural Gas Power Plant 1,2006-06-26,13.7765,121.033
...,...,...,...,...,...,...,...,...,...
414,FDC Misamis Power Corporation,FDC,MINDANAO,GENERATOR,DIRECT,135 MW Coal Fired Thermal Power Plant Unit 1,2020-10-26,8.56104,124.748
415,FDC Misamis Power Corporation,FDC,MINDANAO,GENERATOR,DIRECT,135 MW Coal Fired Thermal Power Plant Unit 2,2020-10-26,8.56104,124.748
416,FDC Misamis Power Corporation,FDC,MINDANAO,GENERATOR,DIRECT,135 MW Coal Fired Thermal Power Plant Unit 3,2020-10-26,8.56104,124.748
417,Malvar Enerzone Corporation,MALVEZ,LUZON,PRIVATE DISTRIBUTION UTILITY,DIRECT,Malvar Enerzone Corporation,2020-10-30,14.0433,121.159


In [366]:
df_wesm.to_csv('/Volumes/data/projects/data/philippine-energy-market/wholesale_participants.csv', index=False)

In [367]:
df_wesm[df_wesm['LON'].isnull()]

,PARTICIPANT NAME,SHORT NAME,REGION,CATEGORY,MEMBERSHIP,RESOURCE,EFFECTIVE DATE,LAT,LON
20,Power Sector Assets & Liabilities Management C...,PSALM,MINDANAO,GENERATOR,DIRECT,80.46 MW Agus I Hydroelectric Power Plant Unit 2,2020-04-25,None,None
159,First Farmers Holding Corporation,FFHC,VISAYAS,GENERATOR,DIRECT,FFHC Co-Generation Bagasse-Fired Power Plant,2010-12-26,None,None
202,Embarcadero Land Ventures Inc.,ELVI,LUZON,BULK USER,INDIRECT,Embarcadero Land Ventures Inc.,2011-11-26,None,None
223,CIP II Power Corporation,CIP2,LUZON,GENERATOR,DIRECT,CIP2 Bunker-Fired Power Plant,2013-01-17,None,None
224,Petron Corporation,PETRONGEN,LUZON,GENERATOR,DIRECT,Circulating Fluidized Bed Power Plant,2013-03-13,None,None
238,AP Renewables Inc. (APRI-Unocal Philippines),APRICST,LUZON,BULK USER,INDIRECT,AP Renewables Inc. (APRI-Unocal Philippines),2014-06-05,None,None
256,Universal Robina Corporation,URC,VISAYAS,GENERATOR,DIRECT,URC Bagasse-Fired Biomass Power Plant,2014-12-18,None,None
258,Majestics Energy Corporation,MEC,LUZON,GENERATOR,DIRECT,Majestics Energy Solar PV Power Plant,2015-01-06,None,None
266,Philippine Radio Educational and Information C...,PREIC-RVA,LUZON,BULK USER,INDIRECT,Philippine Radio Educational and Information C...,2015-04-18,None,None
273,Montalban Methane Power Corp.,MMPC,LUZON,GENERATOR,DIRECT,MMPC Biomass Power Plant,2015-08-13,None,None


In [368]:
df_wesm['LON'].isnull().sum()

25

---

### Retail Market

This section discusses the Retail Market participants.

In [25]:
df_retail.columns

Index(['NAME', 'SHORT NAME', 'REGION', 'CATEGORY', 'MEMBERSHIP',
       'EFFECTIVE DATE', 'STATUS'],
      dtype='object')

In [373]:
df_retail['CATEGORY'].unique()

array(['Contestable Customer', 'Retail Metering Services Provider',
       'Local Retail Electricity Supplier', 'Supplier of Last Resort',
       'Directly Connected Contestable Customer',
       'Retail Electricity Supplier'], dtype=object)

In [372]:
df_retail[df_retail['CATEGORY'] =='Contestable Customer'].head().style.set_table_styles(styles)

,NAME,SHORT NAME,REGION,CATEGORY,MEMBERSHIP,EFFECTIVE DATE
0,Styrotech Corporation,STYR01R,Luzon,Contestable Customer,Indirect,2013-06-26 00:00:00
1,Analog Devices Gen. Trias Inc.,ADGT01R,Luzon,Contestable Customer,Indirect,2013-06-26 00:00:00
2,Analog Devices Gen. Trias Inc.,ADGT02R,Luzon,Contestable Customer,Indirect,2013-06-26 00:00:00
3,Ascott Makati Inc.,ATMK01R,Luzon,Contestable Customer,Indirect,2013-06-26 00:00:00
4,Marina Square Properties Inc.,MSPI01R,Luzon,Contestable Customer,Indirect,2013-06-26 00:00:00


In [382]:
# Removing non-ascii character on line 380
df_retail['NAME'] = df_retail["NAME"].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))

In [383]:
# Reset index since the rows dropped caused the index to skip integer sequence
df_retail = df_retail.reset_index(drop=True)


In [386]:
df_retail.describe

<bound method NDFrame.describe of                                                    NAME SHORT NAME   REGION  \
0                                 Styrotech Corporation    STYR01R    Luzon   
1                        Analog Devices Gen. Trias Inc.    ADGT01R    Luzon   
2                        Analog Devices Gen. Trias Inc.    ADGT02R    Luzon   
3                                    Ascott Makati Inc.    ATMK01R    Luzon   
4                         Marina Square Properties Inc.    MSPI01R    Luzon   
...                                                 ...        ...      ...   
1617                     ANGELICA DL MOJICA CORPORATION    ADLMC1R    Luzon   
1618                              POLYTEX LAIWELL, INC.    PLYTX1R    Luzon   
1619  Dynasty Management and Development Corp. (Gais...    DMDC06R  Visayas   
1620                                Sunpride Foods Inc.    SNPFI1R  Visayas   
1621                   Capiz Electric Cooperative, Inc.  CAPELCMSP  Visayas   

                 

In [387]:
df_retail.isnull().sum()

NAME                 0
SHORT NAME           0
REGION               0
CATEGORY             0
MEMBERSHIP           0
EFFECTIVE DATE       0
LAT               1282
LON               1282
dtype: int64

In [388]:
# Geocoding an address
df_retail["LAT"] = None
df_retail["LON"] = None
country_limit = {'country': 'Philippines'}
geocode_result = []

for i in range(0, len(df_retail.index),1): 
    geocode_result = gmap.geocode(df_retail['NAME'][i],country_limit)
    
    try:
        lat = geocode_result[0]["geometry"]["location"]["lat"]
        lon = geocode_result[0]["geometry"]["location"]["lng"]
        df_retail.iat[i,df_retail.columns.get_loc("LAT")] = lat
        df_retail.iat[i,df_retail.columns.get_loc("LON")] = lon
    except Exception as e:
        #IndexError
        lat = None
        lon = None

In [389]:
df_retail

,NAME,SHORT NAME,REGION,CATEGORY,MEMBERSHIP,EFFECTIVE DATE,LAT,LON
0,Styrotech Corporation,STYR01R,Luzon,Contestable Customer,Indirect,2013-06-26,14.7533,120.99
1,Analog Devices Gen. Trias Inc.,ADGT01R,Luzon,Contestable Customer,Indirect,2013-06-26,14.2632,120.92
2,Analog Devices Gen. Trias Inc.,ADGT02R,Luzon,Contestable Customer,Indirect,2013-06-26,14.2632,120.92
3,Ascott Makati Inc.,ATMK01R,Luzon,Contestable Customer,Indirect,2013-06-26,14.5511,121.026
4,Marina Square Properties Inc.,MSPI01R,Luzon,Contestable Customer,Indirect,2013-06-26,14.5731,120.983
...,...,...,...,...,...,...,...,...
1617,ANGELICA DL MOJICA CORPORATION,ADLMC1R,Luzon,Contestable Customer,Indirect,2020-10-26,14.6512,121.103
1618,"POLYTEX LAIWELL, INC.",PLYTX1R,Luzon,Contestable Customer,Indirect,2020-10-26,14.6392,121.005
1619,Dynasty Management and Development Corp. (Gais...,DMDC06R,Visayas,Contestable Customer,Indirect,2020-10-26,10.3321,123.936
1620,Sunpride Foods Inc.,SNPFI1R,Visayas,Contestable Customer,Indirect,2020-10-26,10.3445,123.952


In [390]:
i

1621

In [391]:
df_retail.to_csv('/Volumes/data/projects/data/philippine-energy-market/retail_participants.csv', index=False)

In [410]:
df_retail_null = df_retail[df_retail['LON'].isnull()]
df_retail_null = df_retail_null.reset_index(drop=True)


In [411]:
df_retail_null.groupby('REGION')['NAME'].nunique()


REGION
Luzon            110
Luzon/Visayas      7
Visayas           14
Name: NAME, dtype: int64

In [412]:
df_retail_null.groupby('CATEGORY')['NAME'].nunique()


CATEGORY
Contestable Customer                       119
Directly Connected Contestable Customer      3
Retail Electricity Supplier                  7
Name: NAME, dtype: int64

In [413]:
df_retail_null[df_retail_null["CATEGORY"]=="Retail Electricity Supplier"]

,NAME,SHORT NAME,REGION,CATEGORY,MEMBERSHIP,EFFECTIVE DATE,LAT,LON
22,Millennium Power RES Inc.,MPRIRES,Luzon/Visayas,Retail Electricity Supplier,Direct,2016-08-18,None,None
28,Corenergy Inc.,CORERES,Luzon/Visayas,Retail Electricity Supplier,Direct,2016-12-23,None,None
100,MeridianX Inc.,MERXRES,Luzon/Visayas,Retail Electricity Supplier,Direct,2019-03-04,None,None
126,Solar Philippines Retail Electricity Inc.,SPREIRES,Luzon/Visayas,Retail Electricity Supplier,Direct,2020-02-18,None,None
133,PHINMA Energy Corporation-RES,PHENRES,Luzon/Visayas,Retail Electricity Supplier,Direct,2013-02-19,None,None
134,Prism Energy Inc.,PRISMRES,Luzon/Visayas,Retail Electricity Supplier,Direct,2013-03-15,None,None
135,Ecozone Power Management Inc.,EPMIRES,Luzon/Visayas,Retail Electricity Supplier,Direct,2013-06-17,None,None


In [ ]:
df.columns = df.columns.str.replace(' ', '_')


In [414]:
df_retail_null['NAME'] = df_retail_null['NAME'].str.strip()  # or .replace as above


In [415]:
#Limited to 2,500 API calls per day
df_retail_null

,NAME,SHORT NAME,REGION,CATEGORY,MEMBERSHIP,EFFECTIVE DATE,LAT,LON
0,Midland Steel Corporation,MSC002R,Luzon,Contestable Customer,Indirect,2013-06-26,None,None
1,Sureste Properties Inc.,SPI001R,Luzon,Contestable Customer,Indirect,2013-12-26,None,None
2,St. Lukes Medical Center Inc.,SLH001R,Luzon,Contestable Customer,Indirect,2014-01-26,None,None
3,Jollibee Worldwide Pte. Ltd.,JWL001R,Luzon,Contestable Customer,Indirect,2014-02-26,None,None
4,Philippine Airlines Inc.,PAL001R,Luzon,Contestable Customer,Indirect,2014-03-26,None,None
...,...,...,...,...,...,...,...,...
151,Philippine Integrated Energy Solutions Inc.,PIES02R,Luzon,Contestable Customer,Indirect,2013-06-26,None,None
152,Maxi Peak Property Development and Management ...,MAXIP1R,Luzon,Contestable Customer,Indirect,2020-08-26,None,None
153,Chuangxing Steel Incorporated,CXSTL1R,Luzon,Contestable Customer,Indirect,2020-08-26,None,None
154,Ground 18 Realty Corporation,GRD181R,Luzon,Contestable Customer,Indirect,2020-10-26,None,None


In [437]:
gmap.geocode(df_retail_null['NAME'][1] +' '+ 'Philippines')

[]

In [435]:
df_retail_null['NAME'][1] +' '+ 'Philippines'

'Sureste Properties Inc. Philippines'

In [429]:
country_limit = {'country': 'Philippines'}